<a href="https://colab.research.google.com/github/Erfan7135/Langchain/blob/main/lang_chain_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!pip install -q langchain langchain-google-genai langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 9.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [2]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"]=userdata.get('GOOGLE_API_KEY')

**import os:** This module provides a way of using operating system dependent functionality, like setting environment variables.

**from google.colab import userdata:** This is a Colab-specific feature to securely access secrets you've stored in the Colab environment.

**os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY'):** This line retrieves your API key from Colab's secret manager (where you should have saved it as GOOGLE_API_KEY) and sets it as an environment variable. LangChain and langchain-google-genai will automatically pick it up from here.

# **Langchain Fundamentals - The Building Blocks**

## 1.1 LLMS

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

response = llm.invoke("1+1=?")

print(response.content)

1+1 = 2


## 1.2 Prompts & Prompts Templates

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} story about a {noun} within 50 words"
)

template = prompt_template.format(adjective="funny",noun="cat")

response = llm.invoke(template)

print(response.content)

Whiskers, a fluffy ginger cat, chased the red laser dot with fierce determination. It bounced, it wiggled! Suddenly, the dot landed square on his human's forehead. Whiskers leaped, headbutting his owner with a triumphant "Mrow!" The human groaned. Whiskers, victorious, then napped on the remote, mission accomplished.


## 1.3 Chains

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# A more advanced prompt template specifically for chat models
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant that translates text."),
    ("user", "Translate the following text into {language}: {text}")
])

translation_chain = chat_prompt_template | llm | StrOutputParser()
# LCEL -> langchain expression language -> compose runnable(?)

result = translation_chain.invoke({"language": "bangla", "text": "I love programming."})

print (result)

আমি প্রোগ্রামিং ভালোবাসি।


## 1.4 Output Parsers

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate

list_parser = CommaSeparatedListOutputParser()

format_instructions = list_parser.get_format_instructions()
#this formate intruction is generated to pass into ai to get dformatted output
print(format_instructions)

prompt = PromptTemplate.from_template(
    "List five {subject}.\n{format_instructions}"
)

list_chain = prompt | llm | list_parser

result = list_chain.invoke({"subject": "ice cream flavors", "format_instructions": format_instructions})

print (result)
print(type(result))

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`
['Vanilla', 'Chocolate', 'Strawberry', 'Mint Chocolate Chip', 'Cookies and Cream']
<class 'list'>


## Assignment

**Your First Assignment:**
Your first task is to combine these concepts.

Task: Create a LangChain chain that:

*   Takes a user_query (e.g., "Summarize this article: [article text]").
*   Uses a ChatPromptTemplate to instruct the Gemini model to summarize the provided text.
*   Pipes the prompt to the llm (your ChatGoogleGenerativeAI instance).
Uses StrOutputParser to get the clean summary.
*   Invoke this chain with a sample article text and print the summary.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

template = ChatPromptTemplate.from_template(
    "Summarize this article: {article_text}"
)

summary_chain = template | llm | StrOutputParser()

article_text = """
The Amazon rainforest is the largest rainforest in the world, covering an area of about 6 million square kilometers.
It spans across nine countries, with the majority of it located in Brazil.
The rainforest is incredibly biodiverse, home to millions of species of plants, animals, and insects, many of which are unique to the region.
It plays a crucial role in regulating the Earth's climate by absorbing vast amounts of carbon dioxide and producing oxygen.
However, the Amazon is under threat from deforestation, primarily due to agricultural expansion, logging, and mining, which has significant implications for global climate change and biodiversity loss.
Efforts are being made to conserve the rainforest, but challenges remain.
"""
summery = summary_chain.invoke({"article_text": article_text})

print(summery)

The Amazon rainforest, the world's largest, covers about 6 million square kilometers across nine countries, primarily Brazil. It is highly biodiverse and vital for global climate regulation by absorbing carbon dioxide and producing oxygen. However, it faces severe threats from deforestation due driven by agricultural expansion, logging, and mining, which has significant implications for global climate change and biodiversity loss, despite ongoing conservation efforts.


# **LangGraph for State & Agentic Systems**


## *2.1 Graph Fundamentals - Nodes & Edges*

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import BaseMessage, HumanMessage

#state
class AgentState(TypedDict):
  messages : Annotated[list[BaseMessage], operator.add]

#graph
workflow = StateGraph(AgentState)

#node
def say_hello(state : AgentState):
  print("Executing 'say_hello' node...")
  current_messages = state.get("messages", [])
  new_message = HumanMessage(content="Hello from the graph")
  current_messages.append(new_message)
  # return {"messages": current_messages}

def say_goodbye(state : AgentState):
  print("Executing 'say_goodbye' node...")
  current_messages = state.get("messages", [])
  new_message = HumanMessage(content="Goodbye from the graph")
  current_messages.append(new_message)
  # return {"messages": current_messages}

#Add node to workflow
workflow.add_node("node_hello", say_hello)
workflow.add_node("node_goodbye", say_goodbye)

workflow.set_entry_point("node_hello")
workflow.add_edge("node_hello", "node_goodbye")
workflow.add_edge("node_goodbye", END)

app = workflow.compile()

print("---Running the simple graph---")
initial_state = {"messages": []}
final_state = app.invoke(initial_state)

for msg in final_state["messages"]:
  print(f"[{msg.type.upper()}]: {msg.content}")

---Running the simple graph---
Executing 'say_hello' node...
Executing 'say_goodbye' node...
[HUMAN]: Hello from the graph
[HUMAN]: Goodbye from the graph


## 2.2 Building LLM Agent

### 2.2.1 Defining Simple Tool

In [ ]:
import os
from langchain_core.tools import tool
from datetime import datetime

@tool
def get_current_datetime() -> str:
  """Returns the current date and time."""
  return "Saturday, June 21, 2025 at 1:17:22 AM +06"

@tool
def get_current_weather(location: str) -> str:
  """Gets the current weather for a specified location."""
  if "dhaka" in location.lower():
    return "It's 30 degrees Celsius and sunny in Dhaka. High humidity."
  elif "london" in location.lower():
      return "It's 15 degrees Celsius and cloudy in London."
  else:
    return "Weather data not available for this location."

print("---Tools Defined---")
print(f"Tool 1 name : {get_current_datetime.name}")
print(f"Tool 1 description : {get_current_datetime.description}")
print(f"Tool 2 name : {get_current_weather.name}")
print(f"Tool 2 description : {get_current_weather.description}")

---Tools Defined---
Tool 1 name : get_current_datetime
Tool 1 description : Returns the current date and time.
Tool 2 name : get_current_weather
Tool 2 description : Gets the current weather for a specified location.


### 2.2.2 Teaching LLM about Tools

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from langchain_core.tools import tool
from datetime import datetime

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

my_tools = [get_current_datetime, get_current_weather]

llm_with_tools = llm.bind_tools(my_tools)

print("--- LLM with Tools Initialized ---")
print("We have bound our tools to the LLM. Now it knows about them.")

from langchain_core.messages import HumanMessage, AIMessage

print("\n--- Invoking LLM to ask for weather ---")
# The LLM doesn't execute the tool here, it just *decides* to call it.
# Its response will contain the tool call suggestion.
response_weather = llm_with_tools.invoke([HumanMessage(content="What's the weather in Dhaka?")])
print(f"LLM's raw response for weather query:")
print(f"Content: {response_weather.content}")
print(f"Tool Calls: {response_weather.tool_calls}")

print("\n--- Invoking LLM to ask for current date and time ---")
response_datetime = llm_with_tools.invoke([HumanMessage(content="What is the current date and time?")])
print(f"LLM's raw response for datetime query:")
print(f"Content: {response_datetime.content}")
print(f"Tool Calls: {response_datetime.tool_calls}")


print("\n--- Invoking LLM for a general question (should NOT call a tool) ---")
response_joke = llm_with_tools.invoke([HumanMessage(content="Tell me a short joke about a dog.")])
print(f"LLM's raw response for joke query:")
print(f"Content: {response_joke.content}")
print(f"Tool Calls: {response_joke.tool_calls}") # This should be an empty list!

NameError: name 'get_current_weather' is not defined

## 2.2.3 Executing Tools and Feeding Results Back

In [11]:
import os
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, END # We'll use END later, just import it now
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from datetime import datetime

# Define our tools again for this section
@tool
def get_current_datetime() -> str:
    """Returns the current date and time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Bundle and bind tools
tools = [get_current_datetime]
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.0)
llm_with_tools = llm.bind_tools(tools)
tool_map = {tool.name: tool for tool in tools} # For looking up

class AgentState(TypedDict):
    messages: Annotated[list[BaseMessage], operator.add]

def call_llm_node(state: AgentState):
    print("Executing 'call_llm_node'...")
    messages = state["messages"]
    response = llm_with_tools.invoke(messages)
    print(f"LLMs response : {response}")
    return {"messages": [response]}

def execute_tool_node(state: AgentState):
    print("Executing 'call_tool_node'...")
    messages = state["messages"]
    tools_called = messages[-1].tool_calls

    if not tools_called:
        return {"messages": []}

    tools_output = []
    for tool_call in tools_called:
        tool_name = tool_call['name']
        tool_args = tool_call['args']
        tool_call_id = tool_call['id']

        print(f"Attempting to execute tool: {tool_name}")

        if tool_name in tool_map:
            called_tool = tool_map[tool_name]
            try:
              output = called_tool.invoke(tool_args)
              tools_output.append(ToolMessage(content=output, tool_call_id=tool_call_id))
            except Exception as e:
              error_message = f"Error executing tool {tool_name}: {str(e)}"
              print(error_message)
              tools_output.append(ToolMessage(content=error_message, tool_call_id=tool_call_id))
        else:
            error_message = f"Error: Unknown tool called: {tool_name}"
            print(error_message)
            tools_output.append(ToolMessage(content=error_message, tool_call_id=tool_call_id))

    print(f"  Tool outputs to add to state: {tools_output}")
    return {"messages": tools_output} # Return the list of ToolMessage objects


workflow = StateGraph(AgentState)

workflow.add_node("llm_node", call_llm_node)
workflow.add_node("tool_node", execute_tool_node)

workflow.set_entry_point("llm_node")

def decide_next_node(state: AgentState):
    print("Executing 'decide_next_node'...")
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        print("Decision: Go to 'tool_node' (LLM wants to use a tool)")
        return "tool"
    else:
        print("Decision: End (LLM provided final answer)")
        return "end"

workflow.add_conditional_edges(
    "llm_node",
    decide_next_node,
    {
        "tool": "tool_node",
        "end": END
    }
)

workflow.add_edge("tool_node", "llm_node")

app = workflow.compile()

print("\n===== TEST 1: Query requiring a tool =====")
user_query_tool = "What is the current date and time?"
initial_state_tool = {"messages": [HumanMessage(content=user_query_tool)]}

print(f"\n--- Initial User Query: '{user_query_tool}' ---")
# Use stream to see what happens at each step (node execution)
for s in app.stream(initial_state_tool):
    print(s)
    if list(s.values()) and 'messages' in list(s.values())[0]:
        final_state_weather = list(s.values())[0]

# To see the final full conversation:
final_conversation_tool = app.invoke(initial_state_tool)
print("\n--- Final Conversation History (Test 1) ---")
for msg in final_conversation_tool['messages']:
    print(f"[{msg.__class__.__name__}]: {msg.content}")
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        print(f"  Tool Calls: {msg.tool_calls}")
    if hasattr(msg, 'tool_call_id') and msg.tool_call_id:
        print(f"  Tool Call ID: {msg.tool_call_id}")


print("\n\n===== TEST 2: Query NOT requiring a tool =====")
user_query_no_tool = "Tell me a fun fact about giraffes."
initial_state_no_tool = {"messages": [HumanMessage(content=user_query_no_tool)]}

print(f"\n--- Initial User Query: '{user_query_no_tool}' ---")
for s in app.stream(initial_state_no_tool):
    print(s)

final_conversation_no_tool = app.invoke(initial_state_no_tool)
print("\n--- Final Conversation History (Test 2) ---")
for msg in final_conversation_no_tool['messages']:
    print(f"[{msg.__class__.__name__}]: {msg.content}")
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        print(f"  Tool Calls: {msg.tool_calls}")



===== TEST 1: Query requiring a tool =====

--- Initial User Query: 'What is the current date and time?' ---
Executing 'call_llm_node'...
LLMs response : content='' additional_kwargs={'function_call': {'name': 'get_current_datetime', 'arguments': '{}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []} id='run--7ff6c6f4-ee85-4525-9af6-b3404775f8eb-0' tool_calls=[{'name': 'get_current_datetime', 'args': {}, 'id': '5efc9bec-97f7-44f8-8911-27fc588b85da', 'type': 'tool_call'}] usage_metadata={'input_tokens': 39, 'output_tokens': 12, 'total_tokens': 90, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 39}}
Executing 'decide_next_node'...
Decision: Go to 'tool_node' (LLM wants to use a tool)
{'llm_node': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_datetime', 'arguments': '{}'}}, response_metadata={'pr

## 2.2.4 The Full Agent Loop

In [15]:
import os
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, END
from typing import Annotated, TypedDict
import operator
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage, ToolMessage
from datetime import datetime

#--- 1. Define Tools ---
@tool
def get_current_datetime():
  """Returns the current Date & Time"""
  return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@tool
def get_current_weather(location: str) -> str:
  """Gets the current weather for a speicified location"""
  if "dhaka" in location.lower():
    return "As of 2025-06-21 13:45:34: It's 30 degrees Celsius and sunny in Dhaka. High humidity."
  elif "london" in location.lower():
      return "It's 15 degrees Celsius and cloudy in London."
  else:
      return "Weather data not available for this location."

tools = [get_current_datetime, get_current_weather]
tool_map = {tool.name: tool for tool in tools}

# --- 2. Initialized llm tools ---
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.0)
llm_with_tools = llm.bind_tools(tools)

# --- 3. Define Agent State ---
class AgentState(TypedDict):
  messages : Annotated[list[BaseMessage], operator.add]

# --- 4. Define Nodes ---

def call_llm_node(state : AgentState):
  print("Executing 'call_llm_node'...")
  messages = state["messages"]
  response = llm_with_tools.invoke(messages)
  return {"messages": [response]}

def execute_tool_node(state : AgentState):
  print("Executing 'execute_tool_node'...")
  messages = state["messages"]
  tools_called = messages[-1].tool_calls

  if not tools_called:
    print("No tool calls to execute")
    return {"messages": []}

  tools_output = []

  for tool_call in tools_called:
    tool_name = tool_call['name']
    tool_args = tool_call['args']
    tool_call_id = tool_call['id']

    print("Attempting to executing tool :{tool_name}")

    if tool_name in tool_map:
      called_tool = tool_map[tool_name]
      try:
        output = called_tool.invoke(tool_args)
        tools_output.append(ToolMessage(content=output, tool_call_id=tool_call_id))
      except Exception as e:
        error_message = f"Error executing tool {tool_name}: {str(e)}"
        print(error_message)
        tools_output.append(ToolMessage(content=error_message, tool_call_id=tool_call_id))
    else:
      error_message = f"Error: Unknown tool called: {tool_name}"
      print(error_message)
      tools_output.append(ToolMessage(content=error_message, tool_call_id=tool_call_id))

  print(f"    Tool outputs to add to state: {tools_output}")
  return {"messages": tools_output}

# --- 5. Define Conditional Edge Logic (The Decision Maker)---
def decide_next_step(state : AgentState):
  print("Executing 'decide_next_step'...")
  messages = state["messages"]
  last_message = messages[-1]

  if last_message.tool_calls:
    print("Decision: Go to 'tool_node' (LLM wants to use a tool)")
    return "tool"
  else:
    print("Decision: End (LLM provided final answer)")
    return "end"

# --- 6. Build  the LangGraph workflow ---

workflow = StateGraph(AgentState)
workflow.add_node("llm_node", call_llm_node)
workflow.add_node("tool_node", execute_tool_node)

workflow.set_entry_point("llm_node")

workflow.add_conditional_edges(
    "llm_node",
    decide_next_step,
    {
        "tool": "tool_node",
        "end": END
    }
)
workflow.add_edge("tool_node", "llm_node")

# --- 7. Compile The Graph ---
app = workflow.compile()

# --- 8. TestCases ---
print("\n===== Running Test Case 1: Query requiring 'get_current_datetime' tool =====")
user_query_datetime = "What is the current date and time?"
initial_state_datetime = {"messages": [HumanMessage(content=user_query_datetime)]}

print(f"\n--- User Query: '{user_query_datetime}' ---")
final_state_datetime = {}
for s in app.stream(initial_state_datetime):
    print(s)
    # The 's' dictionary contains {node_name: {state_update}}.
    # We want to capture the full state from the last step.
    # The 'messages' list is inside the value of the 's' dictionary.
    if list(s.values()) and 'messages' in list(s.values())[0]:
        final_state_datetime = list(s.values())[0]


print("\n--- Final Conversation History (Test 1) ---")
if final_state_datetime:
    for msg in final_state_datetime['messages']:
        print(f"[{msg.__class__.__name__}]: {msg.content.strip()}") # .strip() for cleaner output
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
            print(f"  Tool Calls: {msg.tool_calls}")
        if hasattr(msg, 'tool_call_id') and msg.tool_call_id:
            print(f"  Tool Call ID: {msg.tool_call_id}")
else:
    print("No final state captured for Test 1.")


print("\n\n===== Running Test Case 2: Query requiring 'get_current_weather' tool =====")
user_query_weather = "How's the weather in London?"
initial_state_weather = {"messages": [HumanMessage(content=user_query_weather)]}

print(f"\n--- User Query: '{user_query_weather}' ---")
final_state_weather = {}
for s in app.stream(initial_state_weather):
    print(s)
    if list(s.values()) and 'messages' in list(s.values())[0]:
        final_state_weather = list(s.values())[0]

print("\n--- Final Conversation History (Test 2) ---")
if final_state_weather:
    for msg in final_state_weather['messages']:
        print(f"[{msg.__class__.__name__}]: {msg.content.strip()}")
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
            print(f"  Tool Calls: {msg.tool_calls}")
        if hasattr(msg, 'tool_call_id') and msg.tool_call_id:
            print(f"  Tool Call ID: {msg.tool_call_id}")
else:
    print("No final state captured for Test 2.")


print("\n\n===== Running Test Case 3: Query NOT requiring a tool =====")
user_query_joke = "Tell me a short joke about an elephant."
user_query_joke = "If I go to London from Dhaka, should I bring my umbrella?"
initial_state_joke = {"messages": [HumanMessage(content=user_query_joke)]}

print(f"\n--- User Query: '{user_query_joke}' ---")
final_state_joke = {}
for s in app.stream(initial_state_joke):
    print(s)
    if list(s.values()) and 'messages' in list(s.values())[0]:
        final_state_joke = list(s.values())[0]

print("\n--- Final Conversation History (Test 3) ---")
if final_state_joke:
    for msg in final_state_joke['messages']:
        print(f"[{msg.__class__.__name__}]: {msg.content.strip()}")
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
            print(f"  Tool Calls: {msg.tool_calls}")
else:
    print("No final state captured for Test 3.")


===== Running Test Case 1: Query requiring 'get_current_datetime' tool =====

--- User Query: 'What is the current date and time?' ---
Executing 'call_llm_node'...
Executing 'decide_next_step'...
Decision: Go to 'tool_node' (LLM wants to use a tool)
{'llm_node': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_datetime', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--2f80edbb-c270-4344-88a1-0401d2e2d637-0', tool_calls=[{'name': 'get_current_datetime', 'args': {}, 'id': '1b58b445-bd22-4f91-b28a-6d56a698951e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 82, 'output_tokens': 12, 'total_tokens': 133, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 39}})]}}
Executing 'execute_tool_node'...
Attempting to executing tool :{tool_name}
    Tool outputs to add to st

# **Building a Multi-Tool, Multi-Step Agent for Information Retrieval**

## 3.1 - Enhancing the LLM's Planning with System Messages and Multi-Tool Output

In [16]:
import os
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage, SystemMessage
from datetime import datetime

# Ensure GOOGLE_API_KEY is set up (e.g., from Colab secrets)
# from google.colab import userdata
# os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# --- 1. Define Tools (Same as before) ---
@tool
def get_current_datetime() -> str:
    """Returns the current date and time."""
    # Current time is Saturday, June 21, 2025 at 7:38:06 PM +06.
    return "2025-06-21 19:38:06"

@tool
def get_current_weather(location: str) -> str:
    """Gets the current weather for a specified location."""
    # Current time context: Saturday, June 21, 2025 at 7:38:06 PM +06.
    if "dhaka" in location.lower():
        return "As of 2025-06-21 19:38:06: It's 30 degrees Celsius and partly cloudy in Dhaka. High humidity."
    elif "london" in location.lower():
        return "As of 2025-06-21 19:38:06: It's 18 degrees Celsius and light rain in London. Bring an umbrella."
    elif "paris" in location.lower():
        return "As of 2025-06-21 19:38:06: It's 22 degrees Celsius and sunny in Paris."
    else:
        return "Weather data not available for this location."

tools = [get_current_datetime, get_current_weather]
tool_map = {tool.name: tool for tool in tools}

# --- 2. Initialize LLM with Tools ---
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.0) # Still keeping temp low for predictability
llm_with_tools = llm.bind_tools(tools)

# --- 3. Define Agent State ---
class AgentState(TypedDict):
    messages: Annotated[list[BaseMessage], operator.add]

# --- 4. Define Nodes ---

# MODIFIED call_llm_node to include a SystemMessage
def call_llm_node(state: AgentState):
    print("\n--- Executing 'call_llm_node' ---")
    messages = state["messages"]

    # Add a system message at the beginning of the conversation history
    # This guides the LLM's behavior throughout the interaction
    system_message_content = """You are a helpful and meticulous assistant.
    When asked about weather for multiple locations, use the 'get_current_weather' tool for EACH location mentioned.
    After gathering all necessary information, provide a comprehensive and helpful answer to the user.
    If a tool returns information about bringing an umbrella, explicitly mention that in your final answer.
    """
    # Create a new list for the LLM call, starting with SystemMessage
    # IMPORTANT: The SystemMessage should be at the very beginning of the messages list
    # for the LLM to process it as initial context.
    messages_for_llm = [SystemMessage(content=system_message_content)] + messages

    response = llm_with_tools.invoke(messages_for_llm)
    print(f"LLM Response (from call_llm_node): {response}")
    return {"messages": [response]}

# execute_tool_node (No changes needed, your optimized version is perfect for multiple tool calls!)
def execute_tool_node(state: AgentState):
    print("\n--- Executing 'execute_tool_node' (Optimized) ---")
    messages = state["messages"]
    last_message = messages[-1]

    if not last_message.tool_calls:
        print("Warning: execute_tool_node called, but no tool calls found in last message.")
        return {"messages": []}

    tool_outputs = []
    for tool_call in last_message.tool_calls:
        tool_name = tool_call['name']
        tool_args = tool_call['args']
        tool_call_id = tool_call['id']

        print(f"  Attempting to execute tool: '{tool_name}' with arguments: {tool_args}")

        if tool_name in tool_map:
            called_tool = tool_map[tool_name]
            try:
                output = called_tool.invoke(tool_args)
                tool_outputs.append(ToolMessage(content=output, tool_call_id=tool_call_id))
            except Exception as e:
                error_message = f"Error executing tool '{tool_name}': {e}"
                print(error_message)
                tool_outputs.append(ToolMessage(content=error_message, tool_call_id=tool_call_id))
        else:
            error_message = f"Error: Unknown tool called by LLM: '{tool_name}'"
            print(error_message)
            tool_outputs.append(ToolMessage(content=error_message, tool_call_id=tool_call_id))

    print(f"  Tool outputs to add to state: {tool_outputs}")
    return {"messages": tool_outputs}

# --- 5. Define Conditional Edge Logic (Same as before) ---
def decide_next_step(state: AgentState):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        print("\n--- Decision: LLM wants to use a tool. Routing to 'tool_node'. ---")
        return "tool"
    else:
        print("\n--- Decision: LLM provided final answer. Routing to 'END'. ---")
        return "end"

# --- 6. Build the LangGraph Workflow (Same as before) ---
workflow = StateGraph(AgentState)
workflow.add_node("llm_node", call_llm_node)
workflow.add_node("tool_node", execute_tool_node)
workflow.set_entry_point("llm_node")
workflow.add_conditional_edges(
    "llm_node",
    decide_next_step,
    {
        "tool": "tool_node",
        "end": END
    }
)
workflow.add_edge("tool_node", "llm_node")

# --- 7. Compile the Graph ---
app = workflow.compile()

# --- 8. Test Case for Multi-Location Query ---

print("\n===== Running Test Case: Multi-Location Weather Query =====")
user_query_multi_location = "If I go to London from Dhaka, should I bring my umbrella?"
initial_state_multi_location = {"messages": [HumanMessage(content=user_query_multi_location)]}

print(f"\n--- User Query: '{user_query_multi_location}' ---")
final_state_multi_location = {}
for s in app.stream(initial_state_multi_location):
    print(s)
    if list(s.values()) and 'messages' in list(s.values())[0]:
        final_state_multi_location = list(s.values())[0]

print("\n--- Final Conversation History (Multi-Location Test) ---")
if final_state_multi_location:
    for msg in final_state_multi_location['messages']:
        print(f"[{msg.__class__.__name__}]: {msg.content.strip()}")
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
            print(f"  Tool Calls: {msg.tool_calls}")
        if hasattr(msg, 'tool_call_id') and msg.tool_call_id:
            print(f"  Tool Call ID: {msg.tool_call_id}")
else:
    print("No final state captured for Multi-Location Test.")


===== Running Test Case: Multi-Location Weather Query =====

--- User Query: 'If I go to London from Dhaka, should I bring my umbrella?' ---

--- Executing 'call_llm_node' ---
LLM Response (from call_llm_node): content='' additional_kwargs={'function_call': {'name': 'get_current_weather', 'arguments': '{"location": "London"}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []} id='run--2fec097e-6a4e-481b-a13a-754801969460-0' tool_calls=[{'name': 'get_current_weather', 'args': {'location': 'London'}, 'id': 'f0028de8-cfe2-46b7-b423-a610df8d06de', 'type': 'tool_call'}] usage_metadata={'input_tokens': 161, 'output_tokens': 17, 'total_tokens': 236, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 58}}

--- Decision: LLM wants to use a tool. Routing to 'tool_node'. ---
{'llm_node': {'messages': [AIMessage(content='', additional_kwargs={'function_call

## 3.2: Implementing a Basic ReAct-like Agent

In [17]:
import os
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage, SystemMessage
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Ensure GOOGLE_API_KEY is set up (e.g., from Colab secrets)
# from google.colab import userdata
# os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# --- 1. Define Tools (Same as before) ---
@tool
def get_current_datetime() -> str:
    """Returns the current date and time."""
    return "2025-06-21 19:38:06" # Keeping consistent for reproducibility

@tool
def get_current_weather(location: str) -> str:
    """Gets the current weather for a specified location.
    If the location is not available, state that weather data is not available.
    """
    if "dhaka" in location.lower():
        return "As of 2025-06-21 19:38:06: It's 30 degrees Celsius and partly cloudy in Dhaka. High humidity."
    elif "london" in location.lower():
        return "As of 2025-06-21 19:38:06: It's 18 degrees Celsius and light rain in London. Bring an umbrella."
    elif "paris" in location.lower():
        return "As of 2025-06-21 19:38:06: It's 22 degrees Celsius and sunny in Paris."
    else:
        return "Weather data not available for this location."

tools = [get_current_datetime, get_current_weather]
tool_map = {tool.name: tool for tool in tools}

# --- 2. Initialize LLM with Tools ---
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.0) # Still keeping temp low for predictability

# IMPORTANT: For ReAct, we'll build a custom prompt that includes tool descriptions
# and guides the LLM to output "Thought" and "Action"
# We will bind tools to this *chain*, not directly to the LLM
# The LLM itself will not have tools bound initially; the prompt will describe them.

# --- 3. Define Agent State (Same) ---
class AgentState(TypedDict):
    messages: Annotated[list[BaseMessage], operator.add]

# --- 4. Define Nodes ---

# MODIFIED call_llm_node for ReAct
def call_llm_node(state: AgentState):
    print("\n--- Executing 'call_llm_node' (ReAct Mode) ---")
    messages = state["messages"]

    # This prompt tells the LLM to think step-by-step (Thought)
    # and then decide on an Action (using a tool) or a Final Answer.
    # It dynamically includes the tool descriptions.
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful AI assistant. You have access to the following tools:\n"
                f"{tools}\n\n" # This injects tool descriptions into the prompt
                "You must use the provided tools to answer questions when appropriate.\n"
                "If you need to use a tool, first output your 'Thought' about why you need it, "
                "then output the 'Action' (a tool call). You can make multiple tool calls if needed "
                "to gather all necessary information.\n"
                "If you have gathered all necessary information and can answer the user's query, "
                "output your 'Thought' about the answer, then output a 'Final Answer'.\n\n"
                "Current Conversation History (including tool observations):"
            ),
            ("placeholder", "{messages}") # This will be replaced by the actual message history
        ]
    )

    # Create a runnable that pipes messages through the prompt and then to the LLM
    # We use .bind_tools() on the LLM itself, which uses the model's native function calling
    # The prompt helps guide the *structure* of the LLM's response (Thought/Action)
    # while bind_tools handles the *format* of the tool call JSON.
    llm_chain = prompt | llm.bind_tools(tools)
    response = llm_chain.invoke({"messages": messages})

    print(f"LLM Response (from call_llm_node): {response}")
    return {"messages": [response]}

# execute_tool_node (No changes needed, still perfect for multiple tool calls!)
def execute_tool_node(state: AgentState):
    print("\n--- Executing 'execute_tool_node' (Optimized) ---")
    messages = state["messages"]
    last_message = messages[-1]

    if not last_message.tool_calls:
        print("Warning: execute_tool_node called, but no tool calls found in last message.")
        return {"messages": []}

    tool_outputs = []
    for tool_call in last_message.tool_calls:
        tool_name = tool_call['name']
        tool_args = tool_call['args']
        tool_call_id = tool_call['id']

        print(f"  Attempting to execute tool: '{tool_name}' with arguments: {tool_args}")

        if tool_name in tool_map:
            called_tool = tool_map[tool_name]
            try:
                output = called_tool.invoke(tool_args)
                tool_outputs.append(ToolMessage(content=output, tool_call_id=tool_call_id))
            except Exception as e:
                error_message = f"Error executing tool '{tool_name}': {e}"
                print(error_message)
                tool_outputs.append(ToolMessage(content=error_message, tool_call_id=tool_call_id))
        else:
            error_message = f"Error: Unknown tool called by LLM: '{tool_name}'"
            print(error_message)
            tool_outputs.append(ToolMessage(content=error_message, tool_call_id=tool_call_id))

    print(f"  Tool outputs to add to state: {tool_outputs}")
    return {"messages": tool_outputs}

# MODIFIED decide_next_step for ReAct
def decide_next_step(state: AgentState):
    last_message = state["messages"][-1]

    # Check if the LLM provided a final answer (look for "Final Answer" in its content)
    # This is a heuristic, better parsing might be needed for production
    if "final answer" in last_message.content.lower():
        print("\n--- Decision: LLM provided Final Answer. Routing to 'END'. ---")
        return "end"

    # If not a final answer, it must be a tool call (or an error, which will loop back to LLM eventually)
    elif last_message.tool_calls:
        print("\n--- Decision: LLM wants to use a tool. Routing to 'tool_node'. ---")
        return "tool"
    else:
        # Fallback for unexpected LLM output (e.g., just text but not a final answer)
        # This will send it back to the LLM to try again, perhaps with more context.
        print("\n--- Decision: Unexpected LLM output, looping back to LLM. ---")
        return "llm_node" # Loop back to LLM if it didn't use a tool or give a final answer

# --- 6. Build the LangGraph Workflow (Same Graph Structure) ---
workflow = StateGraph(AgentState)
workflow.add_node("llm_node", call_llm_node)
workflow.add_node("tool_node", execute_tool_node)
workflow.set_entry_point("llm_node")

workflow.add_conditional_edges(
    "llm_node",
    decide_next_step,
    {
        "tool": "tool_node",
        "end": END,
        "llm_node": "llm_node" # Self-loop for unexpected output
    }
)
workflow.add_edge("tool_node", "llm_node") # Always loop back to LLM after tool execution

# --- 7. Compile the Graph ---
app = workflow.compile()

# --- 8. Test Case for Multi-Location Query ---

print("\n===== Running Test Case: Multi-Location Weather Query (ReAct) =====")
user_query_multi_location = "If I go to London from Dhaka, should I bring my umbrella?"
initial_state_multi_location = {"messages": [HumanMessage(content=user_query_multi_location)]}

print(f"\n--- User Query: '{user_query_multi_location}' ---")
final_state_multi_location = {}
for s in app.stream(initial_state_multi_location):
    print(s)
    if list(s.values()) and 'messages' in list(s.values())[0]:
        final_state_multi_location = list(s.values())[0]

print("\n--- Final Conversation History (Multi-Location Test) ---")
if final_state_multi_location:
    for msg in final_state_multi_location['messages']:
        print(f"[{msg.__class__.__name__}]: {msg.content.strip()}")
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
            print(f"  Tool Calls: {msg.tool_calls}")
        if hasattr(msg, 'tool_call_id') and msg.tool_call_id:
            print(f"  Tool Call ID: {msg.tool_call_id}")
else:
    print("No final state captured for Multi-Location Test.")


===== Running Test Case: Multi-Location Weather Query (ReAct) =====

--- User Query: 'If I go to London from Dhaka, should I bring my umbrella?' ---

--- Executing 'call_llm_node' (ReAct Mode) ---
LLM Response (from call_llm_node): content='' additional_kwargs={'function_call': {'name': 'get_current_weather', 'arguments': '{"location": "London"}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []} id='run--d4f17b0a-b885-49a9-865a-23a96c481172-0' tool_calls=[{'name': 'get_current_weather', 'args': {'location': 'London'}, 'id': '8a8f60af-ad5b-4c2b-8f59-7eee44bd1add', 'type': 'tool_call'}] usage_metadata={'input_tokens': 379, 'output_tokens': 17, 'total_tokens': 439, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 43}}

--- Decision: LLM wants to use a tool. Routing to 'tool_node'. ---
{'llm_node': {'messages': [AIMessage(content='', additional_k

Feedback: 


LLM Response (from call_llm_node): content='' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}} id='run--76076e78-8eb1-4d47-9cb9-1265cf54187a-0'

--- Decision: Unexpected LLM output, looping back to LLM. ---
{'llm_node': {'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}}, id='run--76076e78-8eb1-4d47-9cb9-1265cf54187a-0')]}}

--- Executing 'call_llm_node' (ReAct Mode) ---
LLM Response (from call_llm_node): content='' additional_kwargs={'function_call': {'name': 'get_current_weather', 'arguments': '{"location": "London"}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []} id='run--5f0680e9-8e9b-4b4c-b927-176065f0284e-0' tool_calls=[{'name': 'get_current_weather', 'args': {'location': 'London'}, 'id': 'e68c4e34-d737-429a-a1ef-9fec29b35631',

  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 27
}
].


LLM Response (from call_llm_node): content='' additional_kwargs={'function_call': {'name': 'get_current_weather', 'arguments': '{"location": "London"}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []} id='run--9144c0ed-ff3c-4d06-a890-b4b55acdd6cd-0' tool_calls=[{'name': 'get_current_weather', 'args': {'location': 'London'}, 'id': '94d0a8d8-bf2e-4eb1-b2d2-29606cc6549a', 'type': 'tool_call'}] usage_metadata={'input_tokens': 997, 'output_tokens': 17, 'total_tokens': 1073, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 59}}

--- Decision: LLM wants to use a tool. Routing to 'tool_node'. ---
{'llm_node': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_weather', 'arguments': '{"location": "London"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'S

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 25
}
]